<a href="https://colab.research.google.com/github/JohanaR10/ML/blob/main/mml_taller4_finetuning_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mastering Machine Learning 2025

Taller 4: finetuning de modelos de lenguaje para clasificación de texto

Antes de iniciar abra este cuaderno en Google Colab y habilite la ejecución con GPU:
- En el menú Entorno de ejecución seleccione Cambiar tipo entorno de ejecución.
- Asegúrese de tener seleccionado Python 3.
- Como Acelerador de hardware seleccione GPU T4.



Instale las dependencias para asegurar la correcta ejecución del cuaderno.

In [ ]:
!pip install --upgrade datasets evaluate huggingface_hub setfit

Usaremos el dataset de reseñas de pelítuclas de Rotten Tomatoes, disponible en Hugging Face

In [ ]:
from datasets import load_dataset

data = load_dataset('cornell-movie-review-data/rotten_tomatoes')
train_data, test_data = data["train"], data["test"]

## Finetuning de modelos pre-entrenados

Como modelo pre-entrenado usaremos el modelo bert-base-cased, entrenado usando documentos de Wikipedia en inglés.

Note que cargamos el modelo para clasificación de secuencias y definimos el número de etiquetas (labels) igual a 2, lo cual es necesario para crear la red neuronal que se usará sobre el modelo pre-entrenado. También definimos el tokenizador correspondiente al modelo.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_id = "bert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=2
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Creamos un data collator para agregar datos en lotes usando padding para ajustar todas las secuencias a la longitud de la más larga.

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Definimos una función para tokenizar los datos y la usamos en los datos de train y test.

In [ ]:
def preprocess_function(examples):
   """Tokeniza datos de entrada (examples)"""
   return tokenizer(examples["text"], truncation=True)

tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_test = test_data.map(preprocess_function, batched=True)

Definimos una función para calcular las métricas de interés, en este caso el f1-score.

In [ ]:
import numpy as np
import evaluate

def compute_metrics(eval_pred):
   """Calculate F1 score"""
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)

   load_f1 = evaluate.load("f1")
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"f1": f1}

Definimos los objetos para realizar el entrenamiento.

In [ ]:
from transformers import TrainingArguments, Trainer

# Argumentos para el entrenamiento (finetuning) del modelo
training_args = TrainingArguments(
   "model",
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=1,
   weight_decay=0.01,
   save_strategy="epoch",
   report_to="none"
)

# Trainer para realizar el entrenamiento (finetuning)
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

Antes de hacer el finetuning, evaluemos el modelo.

In [ ]:
trainer.evaluate()

Ahora entrenemos y evaluemos el modelo.

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

## Congelar capas

Ahora buscamos hacer un finetuning del modelo BERT, pero congelando algunas de sus capas.

Empezamos definiendo de nuevo el modelo y el tokenizer, como se hizo anteriormente.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=2
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Para explorar las capas del modelo BERT, imprimimos sus nombres

In [ ]:
for name, param in model.named_parameters():
    print(name)

Ahora pasamos a congelar todas las capas, excepto la cabeza de clasificación, cuyo nombre empieza con "classifier". Esto se hace modificando el parámetro requires_grad para que no entre como parte del cálculo de gradiente en el entrenamiento.

In [ ]:
for name, param in model.named_parameters():
     if name.startswith("classifier"):
        param.requires_grad = True
     else:
        param.requires_grad = False

Instanciamos el trainer para ejecutar el entrenamieno

In [ ]:
from transformers import TrainingArguments, Trainer

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


Entrenamos y evaluamos el modelo

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

Ahora buscaremos congelar menos capas para tratar de mejorar el desempeño del modelo.

Cargamos el modelo nuevamente

In [ ]:
model_id = "bert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=2
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Ahora procedemos a congelar todas las capas hasta el bloque encoder 9, dejando las demás habilitadas para entrenamiento. Usamos el índice 165 de los parámetros, que indica el índice del conjunto de parámetros (o layers como aparecen listados anteriormente).

In [ ]:
for index, (name, param) in enumerate(model.named_parameters()):
    if index < 165:
        param.requires_grad = False

Instanciamos y entrenamos el modelo

In [ ]:
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)
trainer.train()

Evaluamos el modelo

In [ ]:
trainer.evaluate()

## Few shot classification

Ahora realizamos la tarea de clasificación usando pocos ejemplos (few shot classification).

Empezamos por seleccionar algunos ejemplos para la tarea, y así simular que tenemos solo 16 casos de cada clase.

In [ ]:
from setfit import sample_dataset

sampled_train_data = sample_dataset(data["train"], num_samples=16)

Note que usamos la librería setfit para esta tarea, la cual requiere que definamos el modelo de embedding que usaremos, en este caso el all-mpnet-base-v2.

In [ ]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained("sentence-transformers/all-mpnet-base-v2")

Ahora definimos los argumentos para el entrenamiento del modelo

In [ ]:
from setfit import TrainingArguments as SetFitTrainingArguments
from setfit import Trainer as SetFitTrainer

args = SetFitTrainingArguments(
    num_epochs=3,     # Número de epocas para el algoritmo de contrastive learning
    num_iterations=20  # Número de parejas de texto a usar
)
args.eval_strategy = args.evaluation_strategy

Instaciamos el entrenador

In [ ]:
trainer = SetFitTrainer(
    model=model,
    args=args,
    train_dataset=sampled_train_data,
    eval_dataset=test_data,
    metric="f1"
)

Entrenamos el modelo. Para entrenar el modelo necesitará crear una cuenta weights and biases (https://wandb.ai), buscar su API key e ingresarla en la caja de ejecución.

In [ ]:
trainer.train()

Evaluamos el modelo

In [ ]:
trainer.evaluate()

## Pre-entrenamiento y finetuning continuo

<font color='red' style="font-size:70px">
Nota: NO ejecute esta sección inicialmente, hágalo solo al final del taller
</font>



Hasta el momento hemos empleado un modelo pre-entrenado al cual le realizamos un finetuning. Ahora consideraremos un pre-enrenamiento adicional del modelo pre-entrenado, el cual usa enmascaramiento  (masked).

Empezamos definiendo el modelo y su tokenizador.

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

model_id = "bert-base-cased"
model = AutoModelForMaskedLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Definimos nuestra función de tokenización y la empleamos para tokenizar los datos de entrenamiento y prueba.

In [ ]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)

tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_train = tokenized_train.remove_columns("label")
tokenized_test = test_data.map(preprocess_function, batched=True)
tokenized_test = tokenized_test.remove_columns("label")

Ahora definimos un data collator para modelado de lenguaje que usa enmascaramiendo (masking), con una probabilidad de enmascarar un token (borrarlo de los datos de entrenamiento para usar como etiquetas).

In [ ]:
from transformers import DataCollatorForLanguageModeling

# Data collator con enmascaramiento
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

Definimos los argumentos de entrenamiento e instanciamos el entrenador.

In [ ]:
training_args = TrainingArguments(
   "model",
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=10,
   weight_decay=0.01,
   save_strategy="epoch",
   report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator
)

Guardamos el tokenizador antes del entrenamiento

In [ ]:
tokenizer.save_pretrained("mlm")

Entrenamos el modelo y lo guardamos

In [ ]:
trainer.train()

model.save_pretrained("mlm")

Para evaluar los modelos, usamos una predicción sobre una frase enmascarada, tal que el modelo realice la predicción de la palabra que debe reemplazar "[MASK]". Primero usamos el modelo pre-entrenado inicial.

In [ ]:
from transformers import pipeline

mask_filler = pipeline("fill-mask", model="bert-base-cased")
preds = mask_filler("What a horrible [MASK]!")

for pred in preds:
    print(f"{pred["sequence"]}")

Ahora cargamos el modelo mlm que preentrenamos con los datos de reseñas de películas y realizamos la misma tarea.

In [ ]:
mask_filler = pipeline("fill-mask", model="mlm")
preds = mask_filler("What a horrible [MASK]!")

for pred in preds:
    print(f"{pred["sequence"]}")

Ahora podemos continuar con la tarea de finetuning, definiendo el modelo como antes.

In [ ]:

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("mlm", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("mlm")

De aquí en adelante los pasos siguen igual que lo hicimos anteriormente.